In [1]:
#FIXME: REWRITE to the latest RAW corpuses format!

In [2]:
#!/usr/bin/env python
# coding: utf8
from __future__ import unicode_literals, print_function

import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding
from tqdm.auto import tqdm

In [3]:
CFG = {'device': 0, 'cpu_count': 4}
TESTS = False
spacy.require_gpu()

True

In [4]:
import gzip, json
def load_entries(fn): # '../data/datasets/nerus.jsonl.gz'
    entries = []
    with gzip.open(fn, 'r') as f:
        for line in tqdm(f):
            entry = json.loads(line)
            entries.append(entry)
    return entries
    #del entries

In [5]:
# from utils.corpus import Corpus

In [6]:
# NERUS_ds_test = load_entries('../../data/datasets/nerus_test.jsonl.gz')
# NERUS_ds_train = load_entries('../../data/datasets/nerus_train.jsonl.gz')
# NERUS = Corpus.from_raw('ru', NERUS_ds_train, NERUS_ds_test)

In [7]:
class Corpus:
    ents = {}
    ds_train = []
    ds_test = []

In [8]:
NERUS = Corpus()
NERUS.ents = {'ORG', 'PER', 'LOC'}
NERUS.ds_test = load_entries('../../data/datasets/nerus_test.jsonl.gz')
NERUS.ds_train = load_entries('../../data/datasets/nerus_train.jsonl.gz')

# BUGFIX for v0.5
# NERUS.ds_test = [{'raw': x['raw'], 'entities': x['entries']} for x in tqdm(NERUS.ds_test)]
# NERUS.ds_train = [{'raw': x['raw'], 'entities': x['entries']} for x in tqdm(NERUS.ds_train)]

# BUGFIX for v0.9
NERUS.ner = NERUS.ents

In [9]:
CORPORA = [NERUS]

In [10]:
for c in CORPORA:
    print(len(c.ner), len(c.ds_train), len(c.ds_test))

3 719295 20000


In [11]:
def get_other_pipes(nlp, x):
    return [pipe for pipe in nlp.pipe_names if pipe != "ner"]

In [12]:
def pluck(dict_list, attr):
    r = []
    for e in dict_list:
        r.append(e[attr])
    return r

def pluck_list(dict_list, *attrs):
    r = []
    for e in dict_list:
        r.append([e[a] for a in attrs])
    return r

def pluck_dict(dict_list, *attrs):
    r = []
    for e in dict_list:
        r.append({a: e[a] for a in attrs})
    return r

In [13]:
# def setup_model():
#     print("Creating new model...")
#     nlp = spacy.blank('ru')
#     return nlp
# nlp = setup_model()
# from tokenizer import set_stemming_tokenizer
# set_stemming_tokenizer(nlp)

In [14]:
import numpy
from spacy.attrs import ID, ORTH, NORM, PREFIX, SUFFIX, SHAPE
from thinc.api import chain, clone, concatenate, with_flatten
from thinc.api import uniqued
from thinc.i2v import HashEmbed
from thinc.misc import FeatureExtracter
from thinc.misc import LayerNorm as LN
from thinc.misc import Residual
from thinc.t2t import ExtractWindow
from thinc.v2v import Model, Maxout


class Vectors(Model):
    name = "fasttext-vectors"

    def __init__(self, storage, vecs, drop_factor=0.0, column=0):
        Model.__init__(self)
        self.storage = storage
        self.vecs = vecs
        self.nV = 300
        self.drop_factor = drop_factor
        self.column = column

    def begin_update(self, ids, drop=0.0):
        if ids.ndim >= 2:
            ids = self.ops.xp.ascontiguousarray(ids[:, self.column])

        s = self.storage[0]
        vectors = numpy.zeros((len(ids), self.nV), dtype=numpy.float32)
        for i, orth in enumerate(ids):
            w = s[int(orth)]
            vectors[i] = self.vecs[w]
        vectors = self.ops.xp.asarray(vectors)
        assert vectors.shape[0] == ids.shape[0]
        return vectors, None


class SaveDoc(Model):
    def __init__(self, storage):
        Model.__init__(self)
        self.storage = storage

    def begin_update(self, docs, drop=0.0):
        for doc in docs:
            if doc.vocab.strings not in self.storage:
                self.storage.append(doc.vocab.strings)
        return docs, _save_doc_bwd


def _save_doc_bwd(d_features, sgd=None):
    return d_features


def my_tok_to_vec(width, embed_size, pretrained_vectors, **kwargs):
    # Circular imports :(
    from spacy._ml import PyTorchBiLSTM

    cnn_maxout_pieces = kwargs.get("cnn_maxout_pieces", 3)
    conv_depth = kwargs.get("conv_depth", 4)
    bilstm_depth = kwargs.get("bilstm_depth", 0)
    cols = [ID, NORM, PREFIX, SUFFIX, SHAPE, ORTH]
    storage = []
    with Model.define_operators({">>": chain, "|": concatenate, "**": clone}):
        # norm = HashEmbed(width, embed_size, column=cols.index(NORM), name="embed_norm")
        # prefix = HashEmbed(
        #     width, embed_size // 2, column=cols.index(PREFIX), name="embed_prefix"
        # )
        # suffix = HashEmbed(
        #     width, embed_size // 2, column=cols.index(SUFFIX), name="embed_suffix"
        # )
        shape = HashEmbed(
            width, embed_size // 2, column=cols.index(SHAPE), name="embed_shape"
        )
        glove = Vectors(storage, pretrained_vectors, width, column=cols.index(NORM), )
        vec_width = glove.nV

        embed = uniqued(
            (glove | shape)
            >> LN(Maxout(width, width + vec_width, pieces=3)),
            column=cols.index(ORTH),
        )

        convolution = Residual(
            ExtractWindow(nW=1)
            >> LN(Maxout(width, width * 3, pieces=cnn_maxout_pieces))
        )

        tok2vec = SaveDoc(storage) >> FeatureExtracter(cols) >> with_flatten(
            embed >> convolution ** conv_depth, pad=conv_depth
        )

        if bilstm_depth >= 1:
            tok2vec = tok2vec >> PyTorchBiLSTM(width, width, bilstm_depth)
        # Work around thinc API limitations :(. TODO: Revise in Thinc 7

        tok2vec.nO = width
        tok2vec.embed = embed
    return tok2vec


In [15]:
import gensim, os
from spacy import util
from spacy._ml import flatten, PrecomputableAffine
from spacy.pipeline.pipes import TextCategorizer, build_simple_cnn_text_classifier, build_bow_text_classifier, \
    build_text_classifier, EntityRecognizer
from spacy.syntax._parser_model import ParserModel
from thinc.api import chain
from thinc.neural import Model
from thinc.v2v import Affine

VECTORS = None
def get_ft_vec():
    global VECTORS
    if VECTORS is None:
        fdir = '../../data/vec/'
        print('loading vectors from', fdir)
        VECTORS = gensim.models.KeyedVectors.load(fdir + 'vectors.bin')
    return VECTORS


def get_t2v(token_vector_width, embed_size, **cfg):
    vectors = get_ft_vec()
    t2v = my_tok_to_vec(token_vector_width, embed_size, vectors)
    return t2v


class MyNER(EntityRecognizer):
    @classmethod
    def Model(cls, nr_class, **cfg):
        depth = util.env_opt('parser_hidden_depth', cfg.get('hidden_depth', 1))
        subword_features = util.env_opt('subword_features',
                                        cfg.get('subword_features', True))
        conv_depth = util.env_opt('conv_depth', cfg.get('conv_depth', 4))
        conv_window = util.env_opt('conv_window', cfg.get('conv_depth', 1))
        t2v_pieces = util.env_opt('cnn_maxout_pieces', cfg.get('cnn_maxout_pieces', 3))
        bilstm_depth = util.env_opt('bilstm_depth', cfg.get('bilstm_depth', 0))
        self_attn_depth = util.env_opt('self_attn_depth', cfg.get('self_attn_depth', 0))
        assert depth == 1
        parser_maxout_pieces = util.env_opt('parser_maxout_pieces',
                                            cfg.get('maxout_pieces', 2))
        token_vector_width = util.env_opt('token_vector_width',
                                          cfg.get('token_vector_width', 96))
        hidden_width = util.env_opt('hidden_width', cfg.get('hidden_width', 64))
        embed_size = util.env_opt('embed_size', cfg.get('embed_size', 2000))
        tok2vec = get_t2v(token_vector_width, embed_size,
                          conv_depth=conv_depth,
                          conv_window=conv_window,
                          cnn_maxout_pieces=t2v_pieces,
                          subword_features=subword_features,
                          bilstm_depth=bilstm_depth)
        tok2vec = chain(tok2vec, flatten)
        tok2vec.nO = token_vector_width
        lower = PrecomputableAffine(hidden_width,
                                    nF=cls.nr_feature, nI=token_vector_width,
                                    nP=parser_maxout_pieces)
        lower.nP = parser_maxout_pieces

        with Model.use_device('cpu'):
            upper = Affine(nr_class, hidden_width, drop_factor=0.0)
        upper.W *= 0

        cfg = {
            'nr_class': nr_class,
            'hidden_depth': depth,
            'token_vector_width': token_vector_width,
            'hidden_width': hidden_width,
            'maxout_pieces': parser_maxout_pieces,
            'pretrained_vectors': None,
            'bilstm_depth': bilstm_depth,
            'self_attn_depth': self_attn_depth,
            'conv_depth': conv_depth,
            'conv_window': conv_window,
            'embed_size': embed_size,
            'cnn_maxout_pieces': t2v_pieces
        }
        return ParserModel(tok2vec, lower, upper), cfg


In [16]:
def create_ner(nlp):
    return MyNER(nlp.vocab)

In [18]:
nlp = spacy.blank('ru')

In [19]:
ner = create_ner(nlp)
if 'ner' in nlp.pipe_names:
    nlp.disable_pipes('ner')
nlp.add_pipe(ner)
nlp.begin_training()
ner(nlp.tokenizer('привет всем'))

loading vectors from ../../data/vec/


привет всем

In [ ]:
# USE_RU2 = False
# if USE_RU2:
#     import ru2e
#     nlp = ru2e.load_ru2('../../ru2_pos_dep_stemming')
# else:
#     nlp = spacy.load('../../ru2')
# nlp.pipeline

In [27]:
def add_ner(nlp, labels, rebuild=True):
    if 'ner' in nlp.pipe_names and rebuild:
        nlp.disable_pipes('ner')
    if 'ner' not in nlp.pipe_names:
        print("Creating new NER...")
        nlp_ner = create_ner(nlp) #nlp.create_pipe('ner')
        nlp.add_pipe(nlp_ner)
    else:
        print("Using existing NER...")
        nlp_ner = nlp.get_pipe('ner')
    print(nlp.pipeline)
    for l in labels:
        nlp_ner.add_label(l)
    assert set(labels) <= set(nlp_ner.labels)
    return nlp_ner
nlp_ner = add_ner(nlp, [l for c in CORPORA for l in c.ents], rebuild=True)
print(nlp_ner.labels)

assert nlp.get_pipe('ner') == nlp_ner

Creating new NER...
[('ner', <__main__.MyNER object at 0x7f503ec77cf8>)]
('PER', 'ORG', 'LOC')


In [28]:
from utils.tqdm import tqdm_batches

In [29]:
def unseen_names(nlp, unseen):
    ner = nlp.get_pipe('ner')
    return [mt for i, mt in enumerate(ner.move_names) if i in unseen]
    
def enable_all_entities(nlp):
    ner = nlp.get_pipe('ner')
    ner.model.unseen_classes = set()

def enable_entities(nlp, labels):
    ner = nlp.get_pipe('ner')
    # print("Unseen classes were:", unseen_names(nlp, ner.model.unseen_classes))
    unseen = set()
    for i, mt in enumerate(ner.move_names):
        if '-' in mt:
            l = mt.split('-', 1)[1]
            # print(mt, l)
            if l not in labels:
                unseen.add(i)
    # print("Set unseen classes to:", unseen_names(nlp, unseen))
    ner.model.unseen_classes = unseen

if TESTS:
    print('Training', KR.ents)
    enable_entities(nlp, KR.ents)

In [30]:
import spacy
import random
from spacy.gold import GoldParse
from spacy.scorer import Scorer
import pandas
pandas.set_option('display.precision', 3) 

def _evaluate(model, batches):
    if model.get_pipe('ner').model is True:
        print("Initializing model!")
        model.begin_training(**CFG)
    scorer = Scorer()
    for batch in batches:
        docs = pluck(batch, 'raw')
        docs = model.pipe(docs)
        for doc, parse in zip(docs, batch):
            scorer.score(doc, GoldParse(doc, entities=parse['entities']))
    return scorer.scores

def evaluate(model, dataset, batch_size=32):
    batches = tqdm_batches(minibatch(dataset, batch_size), total=len(dataset), leave=False)
    return _evaluate(model, batches)

def evaluate_data_source(model, ds, count=None, batch_size=32):
    enable_entities(model, ds.ents)
    if count:
        dataset = random.sample(ds.ds_test, count)
    else:
        dataset = ds.ds_test
    res = evaluate(model, dataset)
    return {k:v for k,v in res['ents_per_type'].items() if k in ds.ents}

def display_ents(list_of_scores):
    display(pandas.DataFrame.from_records(list_of_scores).T)

if TESTS or 0:
    res = evaluate(nlp, NERUS.ds_test, 1000)
    print(pluck_dict([res], 'token_acc', 'tags_acc', 'uas', 'las'))
    display_ents(res['ents_per_type'])

if TESTS or 0:
    scores = evaluate_data_source(nlp, NERUS, count=1000, batch_size=32)
    display_ents(scores)

In [31]:
import numpy

def _train_epoch(model, labels, batches):
    with model.disable_pipes(*get_other_pipes(nlp, 'ner')):
        if model.get_pipe('ner').model is True:
            print("Initializing model!")
            optimizer = model.begin_training(**CFG)
        else:
            optimizer = model.resume_training(**CFG)
        losses = {}
        n_docs = 0
        for batch in batches:
            texts = pluck(batch, 'raw')
            anns = pluck_dict(batch, 'entities')
            enable_entities(model, labels)
            model.update(texts, anns, drop=0.2, losses=losses, sgd=optimizer)
            n_docs += len(batch)
        meta = {
            'docs': n_docs,
            'loss': {k: numpy.log(1e-10 + (v / n_docs)) for k,v in losses.items()},
        }
    enable_all_entities(model)
    return meta

def train_epoch(model, ds, batch_size, count=None):
    if count is None:
        dataset = ds.ds_train.copy()
        random.shuffle(dataset)
    else:
        dataset = random.sample(ds.ds_train, count)
    batches = minibatch(dataset, size=size_)
    _train_epoch(nlp, ds.ents, tqdm_batches(batches, total=len(dataset)))

if TESTS or 0:
    size_ = compounding(1., 32., 1.001)
    train_epoch(nlp, NERUS, batch_size=size_, count=1000)

if TESTS or 0:
    size_ = compounding(1., 32., 1.001)
    train_epoch(nlp, KR, batch_size=size_, count=1000)

if TESTS or 0:
    res = {}
    for c in CORPORA:
        res.update(evaluate_data_source(nlp, c, 1000))
    display_ents(res)

In [32]:
size_ = compounding(1., 32., 1.0005)
for e in tqdm(range(60)):
    for ds_train in CORPORA:
        train_epoch(nlp, ds_train, batch_size=size_, count=1000) # 100000
    res = {}
    for c in CORPORA:
        res.update(evaluate_data_source(nlp, c, 500)) # 5000
    display_ents(res)

Initializing model!


,p,r,f
LOC,81.461,95.149,87.775
ORG,80.949,74.515,77.599
PER,92.626,86.428,89.420


,p,r,f
LOC,89.993,93.558,91.741
ORG,79.832,83.242,81.501
PER,94.802,84.772,89.507


,p,r,f
LOC,94.012,94.638,94.324
ORG,81.968,92.206,86.786
PER,93.686,93.806,93.746


,p,r,f
LOC,96.502,90.990,93.665
ORG,80.410,89.996,84.933
PER,94.628,95.004,94.816


...

,p,r,f
LOC,96.646,97.152,96.898
ORG,92.589,91.346,91.963
PER,96.858,96.905,96.882


,p,r,f
LOC,97.037,97.752,97.393
ORG,92.387,92.197,92.292
PER,97.069,97.329,97.199


,p,r,f
LOC,95.835,97.158,96.492
ORG,94.197,89.185,91.622
PER,97.058,95.566,96.306


,p,r,f
LOC,97.907,97.870,97.888
ORG,90.626,93.326,91.956
PER,97.123,97.941,97.530


,p,r,f
LOC,96.131,97.277,96.701
ORG,92.086,92.160,92.123
PER,97.315,96.776,97.045


In [25]:
#     res = {}
#     for c in CORPORA:
#         res.update(evaluate_data_source(nlp, c, 1000))
#     display_ents(res)

In [26]:
size_ = compounding(1., 32., 1.0005)
for e in tqdm(range(60)):
    for ds_train in CORPORA:
        train_epoch(nlp, ds_train, batch_size=size_, count=1000) # 100000
    res = {}
    for c in CORPORA:
        res.update(evaluate_data_source(nlp, c, 500)) # 5000
    display_ents(res)

Initializing model!


,p,r,f
LOC,56.850,89.806,69.625
ORG,75.013,58.317,65.619
PER,81.473,52.026,63.502


,p,r,f
LOC,80.530,87.645,83.937
ORG,70.128,80.766,75.072
PER,84.792,76.253,80.296


,p,r,f
LOC,88.144,87.267,87.703
ORG,74.670,85.422,79.685
PER,85.099,86.532,85.810


,p,r,f
LOC,89.547,88.647,89.095
ORG,74.469,87.800,80.587
PER,85.516,85.109,85.312


...

,p,r,f
LOC,91.633,90.821,91.225
ORG,83.279,84.466,83.869
PER,85.622,90.398,87.945


,p,r,f
LOC,89.732,92.924,91.300
ORG,84.445,85.954,85.193
PER,88.219,88.219,88.219


,p,r,f
LOC,91.137,90.940,91.039
ORG,84.357,84.890,84.623
PER,85.752,89.532,87.601


,p,r,f
LOC,89.775,92.121,90.933
ORG,82.522,89.185,85.724
PER,89.624,87.396,88.496


KeyboardInterrupt: 

In [33]:
optimizer = nlp.resume_training(**CFG)
with nlp.use_params(optimizer.averages):
    res = evaluate_data_source(nlp, c, count=None)
    display_ents(res)

KeyboardInterrupt: 

In [ ]:
optimizer.averages

In [34]:
import spacy.displacy
def view_example(nlp, s):
    print('Text:', s['raw'])
    doc = nlp(s['raw'])
#     print("Actual:", [(e, e.label_) for e in doc.ents])
    print("Expected:", [(s['raw'][a:b],c,a,b) for a,b,c in s['entities']])
    spacy.displacy.render(doc, style='ent')

enable_all_entities(nlp)

for s in NERUS.ds_test[:2]:
    view_example(nlp, s)
# for s in KR.ds_test[:3]:
#     view_example(nlp, s)

Text: В Москве началось шествие в поддержку кандидата в президенты от "Единой России" Владимира Путина, сообщает корреспондент "Ленты.ру". Колонны двинулись по Фрунзенской набережной в сторону "Лужников", где состоится митинг. На сайте РИА Новости ведется видеотрансляция с мероприятия. "Интерфакс" отмечает, что участники шествия идут по набережной под патриотическую музыку, размахивая флагами. Активисты, по словам заместителя главы предвыборного штаба Путина Алексея Анисимова, заполнили всю Фрунзенскую набережную. "Точным подсчетом занимаются правоохранительные органы", - отметил он. По данным ГУ МВД по Москве, в шествии принимают участие 30 тысяч человек. Когда шествие стартовало, полиция перекрыла проход к месту сбора, отмечает РИА Новости. Очевидцы подтверждают эту информацию в твиттере. "Всех разворачивают. Говорят, что теперь только пешком до Лужи", - пишет, например, пользователь @_Mohandas_. Как заявил агентству  Анисимов, в "Лужниках", тем временем, уже собрались около 10 тысяч

Text: Двукратный олимпийский чемпион по хоккею Владимир Петров назвал кандидатуры пяти тренеров, которые могли бы заменить наставника сборной России Вячеслава Быкова. По словам Петрова, это могли бы сделать Зинэтула Билялетдинов, Андрей Назаров, Федор Канарейкин, Владимир Крикунов или Борис Михайлов, сообщает "Советский спорт". Петров считает, что будет лучше, если главный тренер сборной будет освобожден от работы в клубе и будет заниматься только делами национальной команды. Кроме того, по мнению двукратного олимпийского чемпиона, Федерация хоккея России (ФХР) должна составить контракт тренера таким образом, чтобы в нем были четко прописаны задачи, поставленные перед наставником. Несмотря на то что Петров назвал имена возможных преемников Быкова, он не сказал, должен ли нынешний главный тренер сборной России покинуть свой пост. На прошлой неделе было опубликовано открытое письмо ветеранов российского и советского хоккея с требованием уволить из сборной Игоря Захаркина - помощника Быко

In [35]:
from pathlib import Path
# save model to output directory
def save_model(nlp, output_dir):
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)
save_model(nlp, '../../ru2_ner')

Saved model to ../../ru2_ner


In [37]:
nlp.tokenizer('приветы всем')

приветы всем

In [38]:
spacy.displacy.render(nlp('Вчера мы с Васей пошли гулять по СПб, Гаванская ул. 18'), 
                      style='ent')

In [52]:
spacy.displacy.render(nlp('Вчера мы с Саньком сходили в Макдональдс и пошли гулять по Новосибу, по улице Гаванской 18'), 
                      style='ent')